In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bystro.api import auth, annotation
from bystro.proteomics.annotation_interface import get_annotation_result_from_query, execute_query
import pandas as pd
import json

In [4]:
ancestry_json = None

with open('ancestry_results.json', 'r') as f:
    ancestry_json = json.load(f)

In [5]:
ancestry_df = pd.DataFrame(results)
ancestry_df

NameError: name 'results' is not defined

In [5]:
results = []
for row in ancestry_json['results']:
    # print(row)
    max_superpop_prob = 0
    max_superpop_label = None

    for superpop, value in row['superpops'].items():
        mean = (value['lowerBound'] + value['upperBound']) / 2

        if mean > max_superpop_prob:
            max_superpop_label = superpop
            max_superpop_prob = mean
    
    results.append({
       'sample_id': row['sampleId'],
       'superpop_top_hit': superpop,
       'superpop_top_hit_prob': max_superpop_prob
    })

In [6]:
user = auth.login('akotlar@bu.edu', 'gonxug-xubram-nUbny6', 'https://bystro-dev.emory.edu', print_result=False)
index = '661be6b20f228023b55a6927_657a50d4b2d0278938ba791d'

Existing session found, logging out


In [7]:
results = annotation.get_jobs('completed')
job_id = None
index_name = None
for res in results:
    if 'big_daly' in res.name:
        job_id = res._id
        break
job = annotation.get_jobs(job_id=job_id)

In [8]:
job['outputFileNames']

{'annotation': 'big_daly_vcf-20240414142242561-20240414142242572.annotation.tsv.gz',
 'statistics': {'json': 'big_daly_vcf-20240414142242561-20240414142242572.statistics.json',
  'qc': 'big_daly_vcf-20240414142242561-20240414142242572.statistics.qc.tsv',
  'tab': 'big_daly_vcf-20240414142242561-20240414142242572.statistics.tsv'},
 'config': 'hg38.yml',
 'sampleList': 'big_daly_vcf-20240414142242561-20240414142242572.sample_list',
 'dosageMatrixOutPath': 'big_daly_vcf-20240414142242561-20240414142242572.dosage.feather',
 'header': 'big_daly_vcf-20240414142242561-20240414142242572.annotation.header.json',
 'log': 'big_daly_vcf-20240414142242561-20240414142242572.annotation.log.txt'}

In [9]:
index_name = job['search']['indexName']

In [10]:
job_id

'661be6b20f228023b55a6927'

In [ ]:
import psutil
import os

process = psutil.Process(os.getpid())
print(f"Before run: {process.memory_info().rss / 1e9} GB")

# Default behavior is to show 1 sample per row, e.g. `melt_samples=True`
query_result_df_array_of_structs_select_fields = get_annotation_result_from_query(
    query_string="cadd:>20",
    index_name=index_name,
    bystro_api_auth=user,
    fields=['refSeq.name2', 'refSeq.name', 'refSeq.exonicAlleleFunction', 'refSeq.siteType'],
    explode_field='refSeq.name2',
    tmp_dir='/mnt/ssd1/bystro/python/python/bystro/examples/tmp',
    output_path='foo',
    max_threads=8
)
process = psutil.Process(os.getpid())
print(f"After run: {process.memory_info().rss / 1e9} GB")

Before run: 0.158879744 GB


In [ ]:
import pandas as pd
df = pd.read_feather('0.feather')
df

In [ ]:
df.memory_usage(deep=True).sum()

In [ ]:
df = feather.read_feather('concat.feather', memory_map=True)

In [ ]:
df.shape[0]/1e6

In [ ]:
import psutil
import os
psutil.Process(os.getpid()).memory_info().rss / 1e9

In [ ]:
! du -ah 'concat.feather'

In [ ]:
! du -ah '0.feather'

In [ ]:
concatenate_feather_files_memory_mapped(['0.feather', '0.feather', '0.feather', '0.feather', '0.feather', 
                                         '0.feather', '0.feather', '0.feather', '0.feather', '0.feather', '0.feather', '0.feather'], output_file='concat.feather')

In [ ]:
4358583520/1e6

In [ ]:
pwd

In [ ]:
query_result_df_array_of_structs_select_fields['sample'].apply(lambda x: "FOO" if x == 1847 else "BAR")

In [ ]:
query_result_df_array_of_structs_select_fields.to_csv('query_result_df_array_of_structs_select_fields_exploded.csv')

In [ ]:
query_result_df_array_of_structs_select_fields[query_result_df_array_of_structs_select_fields['locus'] == "chr17:40699291:G:A"]

In [ ]:
merged_df = query_result_df_array_of_structs_select_fields.merge(ancestry_df, left_on='sample', right_on='sample_id')

In [ ]:
# You can choose to also "melt" the dataframe by a field,
# so that when the field has an array value, it is flattened
# When the primary key of the track isn't found, by default we force that 
# field to have 1 value per row, and warn you of this behavior.
# To change it, set `force_flatten_exploded_field=False`
query_result_df = get_annotation_result_from_query(
    query_string="(type:del && (alt:\-6 || alt:\-1)) || (cadd:>40)",
    index_name=index,
    bystro_api_auth=user,
    melt_samples=True,
    explode_field='refSeq.name2',
    fields=['refSeq.name2', 'refSeq.spID', 'refSeq.name'],
    force_flatten_exploded_field=False
)
query_result_df[['refSeq.name2', 'refSeq.spID']]

In [ ]:
# You can disable sample melting by setting `melt_samples=False`
query_result_df = await async_get_annotation_result_from_query(
    query_string="(type:del && (alt:\-6 || alt:\-1)) || (cadd:>40)",
    index_name=index,
    bystro_api_auth=user,
    fields=['refSeq.name2', 'refSeq.name'],
    melt_samples=False,
    explode_field='refSeq.name2'
)
query_result_df.head(n=10)

In [ ]:
# You can also represent tracks that have multiple fields by dictionaries, instead of flattening
# the dictionaries into separate columns, by setting `structs_of_arrays=False`
# This is currently incompatible with `explode_field` and `fields`
query_result_df_array_of_structs_no_melt = get_annotation_result_from_query(
    query_string="*",
    index_name=index,
    bystro_api_auth=user,
    structs_of_arrays=False
)
query_result_df_array_of_structs_no_melt.head(n=10)